## Handwritten Alphanumeric Characters Recognizer

### Code submission for Pre-processing, PCA and Random Forest Classifier (iterations with and without PCA)

The Extended MNIST dataset contains $\small28X28$ images of 62 classes (capital letters, small letters, and numbers).

### Pre processing

In [1]:
#To use the pandas dataframe and functionalities:
import pandas as pd
import numpy as np

#Tried fixing a py4j error that was occuring:
import py4j

#To use pipelines and feature functionality
from pyspark.ml import Pipeline, feature

#To make plots:
import matplotlib.pyplot as plt
import seaborn as sns

#FOr using random forest classifier and to evaluate the performance of the model:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#For reconstruction of the image:
from PIL import Image

#Just to keep the machine awake:
import sys

#For using pyspark:
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn, Row
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
#Making a character list to print the character in the image based on the index. Because the labels are numeric (from 
#0-61) The labels correspond to 0-9 (numbers), uppercase letters and then the lowercase letters.

char_list="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [2]:
#df1=pd.read_csv("emnist-byclass-train.csv")

#Making a column list for all the dataframe; starting with 'label', and then pixel values for all images (1-784):
col_list=["label"]
i=1
while i<785:
    col_list.append("pixel_"+str(i))
    i=i+1
print(col_list)

['label', 'pixel_1', 'pixel_2', 'pixel_3', 'pixel_4', 'pixel_5', 'pixel_6', 'pixel_7', 'pixel_8', 'pixel_9', 'pixel_10', 'pixel_11', 'pixel_12', 'pixel_13', 'pixel_14', 'pixel_15', 'pixel_16', 'pixel_17', 'pixel_18', 'pixel_19', 'pixel_20', 'pixel_21', 'pixel_22', 'pixel_23', 'pixel_24', 'pixel_25', 'pixel_26', 'pixel_27', 'pixel_28', 'pixel_29', 'pixel_30', 'pixel_31', 'pixel_32', 'pixel_33', 'pixel_34', 'pixel_35', 'pixel_36', 'pixel_37', 'pixel_38', 'pixel_39', 'pixel_40', 'pixel_41', 'pixel_42', 'pixel_43', 'pixel_44', 'pixel_45', 'pixel_46', 'pixel_47', 'pixel_48', 'pixel_49', 'pixel_50', 'pixel_51', 'pixel_52', 'pixel_53', 'pixel_54', 'pixel_55', 'pixel_56', 'pixel_57', 'pixel_58', 'pixel_59', 'pixel_60', 'pixel_61', 'pixel_62', 'pixel_63', 'pixel_64', 'pixel_65', 'pixel_66', 'pixel_67', 'pixel_68', 'pixel_69', 'pixel_70', 'pixel_71', 'pixel_72', 'pixel_73', 'pixel_74', 'pixel_75', 'pixel_76', 'pixel_77', 'pixel_78', 'pixel_79', 'pixel_80', 'pixel_81', 'pixel_82', 'pixel_83', 'pi

In [4]:
#Training dataset:
image_df=spark.read.csv("emnist-byclass-train.csv",inferSchema=True,header=True).toDF(*col_list)

In [5]:
#testing dataset:
image_pd_test=pd.read_csv("emnist-byclass-test.csv",header=None)

In [6]:
#Checking column names from the source data (For test data):
label=image_pd_test.iloc[:,0]
#print(label.unique())

print(list(image_pd_test.columns))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [7]:
#image_df.printSchema()

#Renamed column names for training data:
image_df.dtypes

[('label', 'int'),
 ('pixel_1', 'int'),
 ('pixel_2', 'int'),
 ('pixel_3', 'int'),
 ('pixel_4', 'int'),
 ('pixel_5', 'int'),
 ('pixel_6', 'int'),
 ('pixel_7', 'int'),
 ('pixel_8', 'int'),
 ('pixel_9', 'int'),
 ('pixel_10', 'int'),
 ('pixel_11', 'int'),
 ('pixel_12', 'int'),
 ('pixel_13', 'int'),
 ('pixel_14', 'int'),
 ('pixel_15', 'int'),
 ('pixel_16', 'int'),
 ('pixel_17', 'int'),
 ('pixel_18', 'int'),
 ('pixel_19', 'int'),
 ('pixel_20', 'int'),
 ('pixel_21', 'int'),
 ('pixel_22', 'int'),
 ('pixel_23', 'int'),
 ('pixel_24', 'int'),
 ('pixel_25', 'int'),
 ('pixel_26', 'int'),
 ('pixel_27', 'int'),
 ('pixel_28', 'int'),
 ('pixel_29', 'int'),
 ('pixel_30', 'int'),
 ('pixel_31', 'int'),
 ('pixel_32', 'int'),
 ('pixel_33', 'int'),
 ('pixel_34', 'int'),
 ('pixel_35', 'int'),
 ('pixel_36', 'int'),
 ('pixel_37', 'int'),
 ('pixel_38', 'int'),
 ('pixel_39', 'int'),
 ('pixel_40', 'int'),
 ('pixel_41', 'int'),
 ('pixel_42', 'int'),
 ('pixel_43', 'int'),
 ('pixel_44', 'int'),
 ('pixel_45', 'int'),


In [8]:
#Writing a code to bring down the pixel values between 0 to 1; earlier in between 0 to 255:

#Scaling the pixel values from 0-255 to 0-1
scaled_df=image_df
spark_list=col_list[1:]
for c in spark_list:
    name=str(c)
    scaled_df=scaled_df.withColumn(name,fn.col(name)/255)


In [9]:
#Taking a look at the subset of the dataset:

sel=col_list[200:220]
scaled_df.select(*sel).toPandas().head()

,pixel_200,pixel_201,pixel_202,pixel_203,pixel_204,pixel_205,pixel_206,pixel_207,pixel_208,pixel_209,pixel_210,pixel_211,pixel_212,pixel_213,pixel_214,pixel_215,pixel_216,pixel_217,pixel_218,pixel_219
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.074510,0.203922,0.686275,0.976471,0.964706,0.811765,0.356863,0.047059,0.074510,0.603922,0.949020
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007843,0.035294,0.184314,0.623529,0.792157,0.850980,0.870588,0.960784,0.976471,0.960784
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.454902,0.960784,0.996078,0.862745
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015686,0.082353,0.321569,0.505882,0.800000,0.964706,0.996078,0.984314,0.870588,0.843137,0.549020,0.498039,0.447059
4,0.0,0.027451,0.133333,0.686275,0.980392,0.996078,0.968627,0.909804,0.729412,0.803922,0.843137,0.850980,0.850980,0.917647,0.992157,0.996078,0.984314,0.949020,0.494118,0.031373


In [10]:
#Training and validation split:
train_df,validate_df=scaled_df.randomSplit([0.8,0.2],seed=0)

In [ ]:
#Was trying something to find average pixel intensity for every pixel in the image:

#row_mean_df=image_df.withColumn("row_mean",(sum(fn.col(x) for x in image_df.columns[1:])/784))
#avg_intensity=row_mean_df.groupBy("label").agg({"row_mean":"mean"}).withColumnRenamed("avg(row_mean)","average_intensity")
#avg_intensity.first().show()

### Principal Components Analysis

In [11]:
#Vector Assembler:
vect_assembler=feature.VectorAssembler(inputCols=col_list[1:],outputCol="vec_features")

#Principal Component Analysis using all the principal components and making a pipeline:
pca_stage=feature.PCA(k=784,inputCol="vec_features",outputCol="pc")

pipe_pca=Pipeline(stages=[vect_assembler,pca_stage])

In [ ]:
#Fitting the pipeline to make a model
pca_model=pipe_pca.fit(train_df)

In [ ]:
#PCA_Model=pca_model.stages[-1]
#PCA_Model.save("PCA_784_Model")

In [ ]:
#test_pca=feature.PCA.load("PCA_784_Model")

In [ ]:
#Transforming training data using the model created
components_df=pca_model.transform(train_df)

In [ ]:
#Variance explained by every principal component:

pca_stage=pca_model.stages[-1]
variance_list=pca_stage.explainedVariance

### Evaluation

In [ ]:
#Making a dataframe that shows the variance explained by every individual component and also cumilative variance explained:
#That is the variance explained by including all the components till the current one:

pca_pd_df=pd.DataFrame()
pca_pd_df["principal_component_number"]=0
pca_pd_df["variance_explained"]=0
pca_pd_df["cumulative_variance"]=0

comp_num=list(image_pd_test.columns)[:-1]
for i in comp_num:
    pca_pd_df.loc[int(i),"principal_component_number"]=int(i+1)
    pca_pd_df.loc[int(i),"variance_explained"]=float(variance_list[int(i)])
    cumulative_var=0
    for j in range(int(i)+1):
        cumulative_var=cumulative_var+variance_list[j]
    pca_pd_df.loc[int(i),"cumulative_variance"]=cumulative_var

pca_pd_df.principal_component_number=pca_pd_df.principal_component_number.astype("int")
pca_pd_df["percentage_variance_explained"]=(pca_pd_df["cumulative_variance"]/pca_pd_df["variance_explained"].sum())*100
pca_pd_df.head()

### Visualization of PCA

In [ ]:
#print(len(variance_list))
#Plotting PC vs % of variance explained plot and saving it:

sns.set()
ax=sns.lineplot(x="principal_component_number",y="percentage_variance_explained",markers=True,dashes=False,data=pca_pd_df)
ax.set(xlabel="Number of Principal Components",ylabel="Variance Explained (%)")
fig=ax.get_figure()
fig.savefig("PCA.pdf")

In [ ]:
#Plotting PC vs variance explained plot and saving it:

sns.set()
ax=sns.lineplot(x="principal_component_number",y="variance_explained",markers=True,dashes=False,data=pca_pd_df)
ax.set(xlabel="Number of Principal Components",ylabel="Variance Explained")
fig=ax.get_figure()
fig.savefig("PC_var_explained.pdf")

In [ ]:
#Listing the PC vs % variance explained:

variance_explained_list=list(pca_pd_df["percentage_variance_explained"])
num_pca=0
k=0
for i in range(85,100):
    for k in range(len(variance_explained_list)):
        if variance_explained_list[k] < i:
            num_pca=k+1
    print("Variance explained: "+str(i)+"% Number of principals components required: "+str(num_pca))

#184 components are enough to explain 98% of variance: I feel this may be an optimal number; good variance -> reasonable number of componenets
#Will try random forest for 102,119,144,184,259 components.

In [ ]:
pca_df=components_df.select(fn.col("label"),fn.col("pc"))

In [12]:
#Making another PCA for 61 components based on the results from earlier phase:

pca_stage2=feature.PCA(k=61,inputCol="vec_features",outputCol="pc")

pca_pipe2=Pipeline(stages=[vect_assembler,pca_stage2])
pca_model2=pca_pipe2.fit(train_df)

In [ ]:
#pca_model2.save("pca_model_61_pc")

In [13]:
#Similar steps for this PCA model:

components_df2=pca_model2.transform(train_df)

pca_stage2=pca_model2.stages[-1]
variance_list2=pca_stage2.explainedVariance

pca_pd_df2=pd.DataFrame()
pca_pd_df2["principal_component_number"]=0
pca_pd_df2["variance_explained"]=0
pca_pd_df2["cumulative_variance"]=0

comp_num2=list(range(61))
for i in comp_num2:
    pca_pd_df2.loc[int(i),"principal_component_number"]=int(i+1)
    pca_pd_df2.loc[int(i),"variance_explained"]=float(variance_list2[int(i)])
    cumulative_var2=0
    for j in range(int(i)+1):
        cumulative_var2=cumulative_var2+variance_list2[j]
    pca_pd_df2.loc[int(i),"cumulative_variance"]=cumulative_var2

pca_pd_df2.principal_component_number=pca_pd_df2.principal_component_number.astype("int")
pca_pd_df2["percentage_variance_explained"]=(pca_pd_df2["cumulative_variance"]/pca_pd_df2["variance_explained"].sum())*100
pca_pd_df2.head()

,principal_component_number,variance_explained,cumulative_variance,percentage_variance_explained
0,1,0.106319,0.106319,11.827825
1,2,0.068668,0.174987,19.467083
2,3,0.057560,0.232546,25.870528
3,4,0.050775,0.283321,31.519153
4,5,0.044119,0.327441,36.427395


In [ ]:
samp=components_df2.select(fn.col("pc")).first()

In [ ]:
#print(type(samp))
samp

In [ ]:
samp1=components_df.select(fn.col("pc")).first()
samp1

In [ ]:
#optimum_pca=feature.PCA(k=184,inputCol="vec_features",outputCol="pc")
#optimum_pca_pipe=Pipeline(stages=[vect_assembler,optimum_pca])

#optimum_pca_model=optimum_pca_pipe.fit(train_df)

In [ ]:
#Next up we are making multiple random forest models by using different hyperparameters and also evaluating performace for 
#every model in the form of %age of images accurately predicted using validation data and then use the model with best
#performance in validation stage for our usage:

### Random Forest Classifier

In [ ]:
#Making random forest using 61 PCs (90% variance explained), 30 trees with maximum depth of 4
rf1=RandomForestClassifier(featuresCol="pc",labelCol="label",numTrees=30,maxDepth=4,impurity="gini")
rf1_pipe=Pipeline(stages=[pca_model2,rf1])

rf1_model=rf1_pipe.fit(train_df)

In [ ]:
#rf1_model.save("random_forest_k61_30trees_depth4")

In [14]:
mce=MulticlassClassificationEvaluator()

In [ ]:
mce.evaluate(rf1_model.transform(validate_df))

In [ ]:
#Another PCA model with 184 components:
pca_stage3=feature.PCA(k=184,inputCol="vec_features",outputCol="pc")

pca_pipe3=Pipeline(stages=[vect_assembler,pca_stage3])
pca_model3=pca_pipe3.fit(train_df)

In [ ]:
#pca_model3.save("PCA_model_184PC")

In [ ]:
rf2=RandomForestClassifier(featuresCol="pc",labelCol="label",numTrees=30,maxDepth=4,impurity="gini")
rf2_pipe=Pipeline(stages=[pca_model3,rf2])

rf2_model=rf2_pipe.fit(train_df)

In [ ]:
#rf2_model.save("random_forest_184PC_30trees_depth4")

In [ ]:
mce.evaluate(rf2_model.transform(validate_df))

In [ ]:
rf3=RandomForestClassifier(featuresCol="pc",labelCol="label",numTrees=40,maxDepth=5,impurity="gini")
rf3_pipe=Pipeline(stages=[pca_model2,rf3])

rf3_model=rf3_pipe.fit(train_df)

In [ ]:
#rf3_model.save("random_forest_61pc_40trees_depth5")

In [ ]:
val_df=rf4_model.transform(validate_df)
pred_labels=val_df.select(fn.col("label"),fn.col("prediction")).toPandas()

In [ ]:
pred_labels=pred_labels.astype({"label":int,"prediction":int})
pred_labels.head()

In [ ]:
#Making a confusion matrix. Using scikit learn to make the matrix cause it is easier

from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(pred_labels["label"], pred_labels["prediction"])
#print(conf_mat)
conf_mat_df=pd.DataFrame(conf_mat)

In [ ]:
#conf_mat_df.head()
idx=list(char_list)
conf_mat_df.columns=idx
conf_mat_df.index=idx
conf_mat_df.head()

In [ ]:
sns.set()
ax = sns.heatmap(conf_mat_df)
fig=ax.get_figure()
fig.savefig("RF_conf_matrix.pdf")

In [ ]:
mce.evaluate(rf3_model.transform(validate_df))

In [15]:
rf4=RandomForestClassifier(featuresCol="pc",labelCol="label",numTrees=50,maxDepth=6,impurity="gini")
rf4_pipe=Pipeline(stages=[pca_model2,rf4])

rf4_model=rf4_pipe.fit(train_df)

In [ ]:
#rf4_model.save("random_forest_61pc_50trees_depth6")

In [16]:
mce.evaluate(rf4_model.transform(validate_df))

0.4107110031073609

In [ ]:
rf5_pipe=Pipeline(stages=[pca_model3,rf3])

rf5_model=rf5_pipe.fit(train_df)

In [ ]:
mce.evaluate(rf5_model.transform(validate_df))

In [ ]:
#This configuration doesn't work
rf6_pipe=Pipeline(stages=[pca_model3,rf4])

rf6_model=rf6_pipe.fit(train_df)

In [ ]:
mce.evaluate(rf6_model.transform(validate_df))

In [ ]:
#This config does not work:

#rf5=RandomForestClassifier(featuresCol="pc",labelCol="label",numTrees=60,maxDepth=6,impurity="gini")
#rf5_pipe=Pipeline(stages=[pca_model2,rf5])

#rf5_model=rf5_pipe.fit(train_df)

In [ ]:
#mce.evaluate(rf5_model.transform(validate_df))

In [ ]:
#To look at the frequency of the distribution of characters across all the images:
freq_df=image_df.groupBy("label").count()
freq_pd_df=freq_df.toPandas().sort_values("count",ascending=False)

In [ ]:
#Associating label to its respective frequency:

freq_pd_df["char"]="-"
i=0
while i<len(freq_pd_df):
    freq_pd_df.loc[i,"char"]=char_list[int(freq_pd_df.loc[i,"label"])]
    i=i+1
freq_pd_df

In [ ]:
#Plotting the graph showing frequency distribution:

ax=sns.barplot(x="char",y="count",data=freq_pd_df)
ax.set(xlabel="Character",ylabel="Frequency")
ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
fig=ax.get_figure()
fig.savefig("count_dist.pdf")

Following part is the raw input function:

### Live Testing

In [48]:
#In this testing function, we are reading the image (handwritten by the user) and bring it to the format readable by our model
#The data is mad einto a dataframe as a column initially so transposed it to make it into a row wise dataframe:

img=Image.open("test_1.png").convert("1")
np_img=np.array(img.getdata())

img_pd=pd.DataFrame(np_img).T
img_pd.columns=col_list[1:]

In [49]:
#Scaling the pixel values from 0-255 to 0-1:

img_sp=spark.createDataFrame(img_pd)
scaled_test_df=img_sp
spark_test_list=col_list[1:]mlp_
for c in spark_test_list:
    name=str(c)
    scaled_test_df=scaled_test_df.withColumn(name,fn.col(name)/255)

In [51]:
#Finally using the best random forest model to get a prediction:
test_df=rf4_model.transform(scaled_test_df)

In [53]:
pred_df=test_df.select(fn.col("prediction")).toPandas()

In [54]:
#Based on the label prediction, printing the actual character from char_list:

prediction=pred_df.iloc[0,0]
print(char_list[int(prediction)])

3


In [40]:
arr=np.array(img_pd.iloc[0,:])
np.array_equal(np_img,arr)

True